# 2D landscape evolution model (bedrock channel SPL + hillslope diffusion)

The code below runs a simple landscape evolution model using `fastscape`. It starts from extracted elevation data (SRTM) using the [bmi-topography](https://github.com/csdms/bmi-topography) package.

## Goal

- Leverage different Python tools (connecting the dots) to create an interactive, nice-looking dashboard.
- We'll use `ipywidgets`, `ipyleaflet` and `ipygany`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xsimlab as xs

from bmi_topography import Topography
from fastscape.models import basic_model
from fastscape.processes import SurfaceTopography

In [ ]:
def extract_dem(north=40.25, south=39.75, west=-105.25, east=-104.75):
    """Extract elevation data using a region of interest."""
    
    params = {
        'dem_type': 'SRTMGL3',
         'south': south,
         'north': north,
         'west': west,
         'east': east,
         'output_format': 'GTiff',
         'cache_dir': '~/.bmi_topography'
    }
    
    topo = Topography(**params)

    print("Fetch topographic data...")
    topo.fetch()

    print("Load dataset...")
    return topo.load()

In [ ]:
@xs.process
class InitDEM:
    dem = xs.variable(dims=('y', 'x'))
    elevation = xs.foreign(SurfaceTopography, 'elevation', intent='out')
    
    def initialize(self):
        self.elevation = self.dem
        

model = basic_model.update_processes({'init_topography': InitDEM})


def run_model(dem, U=1e-5, K=2e-6, m=0.6, n=1.5, D=1e-2):
    
    resolution = 90
    grid_shape = [dem.sizes['y'], dem.sizes['x']]
    grid_length = list(np.array(grid_shape) * resolution)
    print(grid_length)

    in_ds = xs.create_setup(
        model=model,
        clocks={
            'time': np.arange(0, 1e6 + 1e4, 2e4),
            'out': np.arange(0, 1e6 + 1e4, 2e4),
        },
        master_clock='time',
        input_vars={
            'grid__shape': grid_shape,
            'grid__length': grid_length,
            'boundary__status': 'fixed_value',
            'init_topography__dem': dem.squeeze().astype('d'),
            'uplift__rate': U,
            'spl': {
                'k_coef': K,
                'area_exp': m,
                'slope_exp': n
            },
            'diffusion__diffusivity': D
        },
        output_vars={
            'topography__elevation': 'out',
        }
    )
    
    print("Starting landscape evolution model run...")
    
    with xs.monitoring.ProgressBar():
        out_ds = in_ds.xsimlab.run(model=model)
    
    return out_ds

In [ ]:
dem = extract_dem()
out_ds = run_model(dem)
            

In [ ]:
out_ds

In [ ]:
out_ds.topography__elevation.isel(out=-1).plot(aspect=1, size=10);

---

## Exercise 1

Create a map widget that we can use to select a region of interest that is then used to extract DEM data and run a simulation.

Hints:

- use `ipywidgets.DrawControl` to draw a ROI (rectangle) and get the its coordinates
- create a `Button` widget with a callback function that will get the ROI coordinates, download/load DEM data and run the simulation
- create an `Output` widget to log the simulation progress and show the resulting Xarray Dataset

## Exercise 2

Let's visualize the final elevation (model output).

Hints:

- create a new `Output` widget that will contain an `ipygany.Scene` widget
- create the ipygany scene using `ipygany.Polymesh` and `ipygany.IsoColor`.
    - you can get the final elevation using `ds_out.topography__elevation.isel(out=-1)` from the output xarray Dataset
    - do `from utils import get_mesh_elements` and use this utility function to get the elevation array as an unstructured mesh (needed by `ipygany.Polymesh`)

Bonuses:

- add widgets for setting custom model parameter values
- animate the model outputs in the ipygany scene (there's an example here: https://nbviewer.jupyter.org/github/QuantStack/ipygany/blob/master/examples/ipygany.ipynb) 